In [1]:
import os
import pandas as pd

In [2]:
pd.options.display.float_format = "{:,.0f}".format

In [4]:
# Paths
ppp_path = "../data/PPP_data_150k_plus.csv"
naics_path = "../data/2017_NAICS_Structure.xlsx"

#Load Data into DF's
naics_df = pd.read_excel(naics_path, header=2)
ppp = pd.read_csv(ppp_path)

In [5]:
data_dir = "../data"

In [6]:
cols = ["LoanAmount", "Zip", "State", "NAICSCode", "BusinessType", "JobsReported", "DateApproved"]
df = pd.DataFrame(columns = cols)
for folder in os.listdir(os.path.join(data_dir, "All Data 0808")):
    if folder.endswith(".pdf"): continue
    for file in os.listdir(os.path.join(data_dir, "All Data 0808", folder)):
        if file.endswith(".csv"):
            if file == "PPP Data 150k plus 080820.csv": continue
            df = df.append(pd.read_csv(os.path.join(data_dir, "All Data 0808", folder, file), usecols = cols))

### NAICS Merge w/ PPP Data

In [7]:
test = ppp[ppp["NAICSCode"].isna() == False]
test.shape
df_new = test.merge(naics_df, how="left", left_on="NAICSCode", right_on="2017 NAICS Code")

### PPP Data Merge w/ States that are present in yelp data

In [8]:
state_ls = ["AZ", "NV", "OH", "NC", "PA", "WI", "IL", "SC"]
df_merge = df_new[df_new["State"].isin(state_ls)]

In [9]:
# Rename columns to match column names in df_new
cols2 = ["LoanAmount", "Zip", "State", "NAICSCode", "BusinessType", "JobsReported", "DateApproved", "2017 NAICS Title"]
df_merge = df_merge[["LoanRange", "Zip", "State", "NAICSCode", "BusinessType", "JobsRetained", "DateApproved", "2017 NAICS Title"]]
df_merge.columns = cols2
df_merge.head()

,LoanAmount,Zip,State,NAICSCode,BusinessType,JobsReported,DateApproved,2017 NAICS Title
13773,a $5-10 million,"85,018",AZ,"621,111",Limited Liability Company(LLC),438,04/27/2020,Offices of Physicians (except Mental Health Sp...
13774,a $5-10 million,"85,205",AZ,"524,130",Corporation,2,04/28/2020,Reinsurance Carriers
13775,a $5-10 million,"85,203",AZ,"523,999",Subchapter S Corporation,471,04/27/2020,Miscellaneous Financial Investment Activities
13776,a $5-10 million,"85,711",AZ,"621,111",Corporation,500,04/29/2020,Offices of Physicians (except Mental Health Sp...
13777,a $5-10 million,"85,016",AZ,"721,110",Partnership,0,04/30/2020,Hotels (except Casino Hotels) and Motels


In [12]:
# Merge codes onto data with loans under 150k loans
test2 = df[df["NAICSCode"].isna() == False]
print(df.shape)
df_new2 = test2.merge(naics_df, how="left", left_on="NAICSCode", right_on="2017 NAICS Code")
df_new2.head()

(836028, 7)


,LoanAmount,Zip,State,NAICSCode,BusinessType,JobsReported,DateApproved,Change Indicator,2017 NAICS Code,2017 NAICS Title
0,"149,990",17070,PA,"721,110",Corporation,29,04/13/2020,NaN,721110,Hotels (except Casino Hotels) and Motels
1,"149,989",17754,PA,"332,710",Corporation,0,08/04/2020,NaN,332710,Machine Shops
2,"149,987",19610,PA,"541,310",Subchapter S Corporation,11,04/15/2020,NaN,541310,Architectural Services
3,"149,985",17111,PA,"237,990",Subchapter S Corporation,16,04/14/2020,NaN,237990,Other Heavy and Civil Engineering Construction
4,"149,977",19124,PA,"541,511",Limited Liability Company(LLC),20,08/06/2020,NaN,541511,Custom Computer Programming Services


In [11]:
# Save new dataframes to csv
df_new2.to_csv("under150k.csv")
df_merge.to_csv("over150k.csv")